In [8]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

path1 = Path('../../../czl/TwiBot22-baselines/datasets/Twibot-20')
path2 = Path('../../../czl/TwiBot22-baselines/data')
path3 = Path('../../../czl/TwiBot22-baselines/src')


In [88]:
def str_to_bool(label):
    if label[1] == 'human':
        return (label[0], 1)
    else:
        return (label[0], 0)

In [2]:

node_info = pd.read_json(path1 / 'node.json')
label = pd.read_csv(path1 / 'label.csv')
split = pd.read_csv(path1 / 'split.csv')
node_info = pd.merge(node_info, label)
node_info = pd.merge(node_info, split)


In [89]:

node_des = node_info['description']
node_name = node_info['username']
node_label = node_info['label']
node_split = node_info['split']

node_des = dict(map(lambda x: x, enumerate(node_des)))
node_name = dict(map(lambda x: x, enumerate(node_name)))
node_label = dict(map(str_to_bool, enumerate(node_label)))
node_split = dict(map(lambda x: x, enumerate(node_split)))

np.save('data/node_des.npy', node_des)
np.save('data/node_name.npy', node_name)
np.save('data/node_label.npy', node_label)
np.save('data/node_split.npy', node_split)



In [34]:
"""
below name->tensor
"""
from transformers import RobertaTokenizer, RobertaModel, pipeline
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
feature_extractor = pipeline('feature-extraction', model=RobertaModel.from_pretrained(model_name), tokenizer=tokenizer, device=0)

node_name = np.load('data/node_name.npy', allow_pickle=True).tolist()

names_vector = []
for i in tqdm(range(len(node_name))):
    name_vector = torch.tensor(feature_extractor(node_name[i])).squeeze(0)
    name_vector = torch.mean(name_vector, dim=0)
    names_vector.append(name_vector)
    
torch.stack(names_vector, dim=0)
torch.save(names_vector, 'data/names_vector.pt')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
"""
note: len(users_tweets == 229580) , len(node_des == 11826)
"""
des_vector = torch.load(path2 / 'des_tensor.pt')
names_vector = torch.load('data/names_vector.pt')
users_tweets = np.load(path3 / 'T5/Twibot-20/data/user_tweets_dict.npy', allow_pickle=True).tolist()


In [102]:
"""
concat text
"""
users_text = {}
for i in tqdm(range(len(node_des))):
    user_tweets = ''.join(users_tweets[i])
    users_text[i] = '. '.join([node_name[i].strip(' '), node_des[i].strip(' '), user_tweets])
    
np.save('data/users_text.npy', users_text)

100%|██████████| 11826/11826 [00:00<00:00, 19081.38it/s]
